In [1]:
from data.points import TAG, ANCHORS
import math

static = True
# filename = None
filename = "tmp"



if static:
    def location():
        while True:
            # yield (*TAG["P1"], 1000)
            yield (1500,1500, 0)
else:
    def location():
        """Circle generator"""
        angle = 0.0
        radius = 1000
        center = (1500,1500)
        height = 0
        while True:
            x = radius * math.cos(angle) + center[0]
            y = radius * math.sin(angle) + center[1]
            angle += 0.01
            yield x, y, height
    

    def location(side_length, height, step_size, center_x=0, center_y=0):
        half_side_length = side_length / 2
        x = center_x - half_side_length
        y = center_y - half_side_length

        # Move along the bottom side
        while x < center_x + half_side_length:
            yield (x, y, height)
            x += step_size

        # Move along the right side
        while y < center_y + half_side_length:
            yield (x, y, height)
            y += step_size

        # Move along the top side
        while x > center_x - half_side_length:
            yield (x, y, height)
            x -= step_size

        # Move along the left side
        while y > center_y - half_side_length:
            yield (x, y, height)
            y -= step_size



In [2]:
samples_per_anchor = 1000
# sigma = 250

In [3]:
# import matplotlib.pyplot as plt
# data = {"x": [], "y": [], "z": []}
# circle = location()
# for _ in range(samples_per_anchor):
#     x,y,z = next(circle)
#     print(x,y,z)
#     data["x"].append(x)
#     data["y"].append(y)
#     data["z"].append(z)
# plt.plot(data["x"], data["y"])

In [4]:
from math import sqrt

def p2p_dist(ax, ay, az, bx, by, bz):
    return sqrt((bx - ax) ** 2 + (by - ay) ** 2 + (bz-az) ** 2)

In [5]:
# Get std from some data
import pandas as pd

stationary_data = "data/p1/2023-07-04-21:59:08.884069living-room-p1-7.csv"
ref_df = pd.read_csv(stationary_data)
sigma = ref_df[["distance", "anchor"]].groupby("anchor").std().mean()

In [6]:
import numpy as np
from datetime import datetime, timedelta
from itertools import cycle

points = location(3500, 0, 20, TAG["P3"][0]/2, TAG["P3"][1]/2)

# Unit: mm
# simulate_data = lambda p1, p2: np.random.normal(p2p_dist(*p1, *p2), sigma, samples_per_anchor)
data = {"anchor": [], "distance": [], "real_x": [], "real_y": []}
# data2 = {"anchor": [], "distance": []}
# for k, v in ANCHORS.items():
#     data["anchor"].append(k)
#     data["distance"].append(simulate_data((v[0], v[1], 0), next(points)))

# for round in zip(*data["distance"]):
#     for row in zip(data["anchor"], round):
#         data2["anchor"].append(row[0])
#         data2["distance"].append(row[1])

i = 0
try:
    for anchor, anchor_pos in cycle(ANCHORS.items()):
        current_position = next(points)
        distance = np.random.normal(p2p_dist(*current_position, *anchor_pos, 0), sigma, size=None)[0]
        data["anchor"].append(anchor)
        data["distance"].append(distance)
        data["real_x"].append(current_position[0])
        data["real_y"].append(current_position[1])

        if i >= len(ANCHORS) * samples_per_anchor:
            break
        i += 1
except StopIteration:
    pass



df = pd.DataFrame(data)
df["los_confidence"] = 1
# TODO fix acc
df["acc_x"] = np.random.normal(0, sigma, len(df.index))
df["acc_y"] = np.random.normal(0, sigma, len(df.index))
df["acc_z"] = np.random.normal(0, sigma, len(df.index))
dt = datetime.now()
df["timestamp"] = [dt + timedelta(milliseconds=i*200) for i in range(len(df.index))]
# ,anchor,distance,instant,timestamp,acc_x,acc_y,acc_z,los_confidence,reference_distance

generated_filename = f"data/simulated/{datetime.now().isoformat()}.csv"
df.to_csv(f"data/simulated/{filename}.csv" or generated_filename)
df

TypeError: location() takes 0 positional arguments but 5 were given